In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import gridtools as gt
import thunderfish as tf
from plotstyle import PlotStyle
%matplotlib qt
s = PlotStyle()

In [2]:
# path to recordings
path1 = "/home/weygoldt/Data/uni/efish/output/2016-04-09-22_25/"
path2 = "/home/weygoldt/Data/uni/efish/output/2016-04-16-18_45/"
path3 = "/home/weygoldt/Data/uni/efish/output/2016-04-20-18_49/"
path4 = "/home/weygoldt/Data/uni/efish/output/2016-04-18-19_22/"

# grid instances
grid1 = gt.GridTracks(path1, finespec=True)
grid2 = gt.GridTracks(path2, finespec=True)
grid3 = gt.GridTracks(path3, finespec=True)
grid4 = gt.GridTracks(path4, finespec=True)

# event dataframes
events1 = pd.read_csv(path1 + "events.csv")
events2 = pd.read_csv(path2 + "events.csv")
events3 = pd.read_csv(path3 + "events.csv")
events4 = pd.read_csv(path4 + "events.csv")


# selected event indices
sel_events1 = 1
sel_events2 = 2
sel_events3 = 7
sel_events4 = 1

[ GridTracks.__init__ ] No temperature an light data found in directory /home/weygoldt/Data/uni/efish/output/2016-04-09-22_25/.
[ GridTracks.__init__ ] No grid metadata found in directory /home/weygoldt/Data/uni/efish/output/2016-04-09-22_25/
[ GridTracks.__init__ ] No grid metadata found in directory /home/weygoldt/Data/uni/efish/output/2016-04-16-18_45/
[ GridTracks.__init__ ] No grid metadata found in directory /home/weygoldt/Data/uni/efish/output/2016-04-20-18_49/
[ GridTracks.__init__ ] No grid metadata found in directory /home/weygoldt/Data/uni/efish/output/2016-04-18-19_22/


In [3]:
# extract needed indices and dyads

timepad = 60

ids1 = [int(events1.id1[sel_events1]), int(events1.id2[sel_events1])]
dyad1 = gt.Dyad(grid1, ids1)
idxs1 = [
    gt.utils.find_closest(dyad1.times, events1.start[sel_events1]-timepad),
    gt.utils.find_closest(dyad1.times, events1.stop[sel_events1]+timepad)
]
newtime1 = (dyad1.times - events1.start[sel_events1])[idxs1[0]:idxs1[1]]



ids2 = [int(events2.id1[sel_events2]), int(events2.id2[sel_events2])]
dyad2 = gt.Dyad(grid2, ids2)
idxs2 = [
    gt.utils.find_closest(dyad2.times, events2.start[sel_events2] -timepad),
    gt.utils.find_closest(dyad2.times, events2.stop[sel_events2] +timepad)
]
newtime2 = (dyad2.times - events2.start[sel_events2])[idxs2[0]:idxs2[1]]



ids3 = [int(events3.id1[sel_events3]), int(events3.id2[sel_events3])]
dyad3 = gt.Dyad(grid3, ids3)
idxs3 = [
    gt.utils.find_closest(dyad3.times, float(events3.start[sel_events3]) - timepad-20),
    gt.utils.find_closest(dyad3.times, float(events3.stop[sel_events3]) + timepad+20)
]
newtime3 = (dyad3.times - events3.start[sel_events3])[idxs3[0]:idxs3[1]]



ids4 = [int(events4.id1[sel_events4]), int(events4.id2[sel_events4])]
dyad4 = gt.Dyad(grid4, ids4)
idxs4 = [
    gt.utils.find_closest(dyad4.times, float(events4.start[sel_events4])-timepad),
    gt.utils.find_closest(dyad4.times, float(events4.stop[sel_events4])+timepad)
]
newtime4 = (dyad4.times - events4.start[sel_events4])[idxs4[0]:idxs4[1]]



[ Dyad.__init__  ] GridTracks instance has no temperature and light arrays.


In [4]:
from matplotlib.gridspec import GridSpec

# Grid setup
hr = [1]
wr = [0.4, 0.6, 1]
ny = len(hr)
nx = len(wr)
fig = plt.figure(figsize=(300 * s.mm, 70 * s.mm), constrained_layout=False)

grid = GridSpec(ny, nx, figure=fig, height_ratios=hr, width_ratios=wr)

grid.update(left=0.06, right=0.98, bottom=0.18, top=0.93, wspace=0.25, hspace=0.2)

ax1 = fig.add_subplot(grid[0, 0])
ax2 = fig.add_subplot(grid[0, 1])
ax3 = fig.add_subplot(grid[0, 2])

ax1.set_xlabel('time [s]')
ax2.set_xlabel('time [s]')
ax3.set_xlabel('time [s]')
ax1.set_ylabel("EOD$f$  [Hz]", size = 14)

# plot spectra
# get y limits for spectrogram

def plot_finespec(ax, idxs, dyad, newtime, grid, style):
    
    xlims = [dyad.times[idxs[0]], dyad.times[idxs[1]]]

    ylims = gt.utils.get_ylims(
        dyad.fund_id1, dyad.fund_id2, dyad.times, xlims[0], xlims[1]
    )

    # get axes limits
    f0, f1 = ylims[0], ylims[1]
    t0, t1 = xlims[0], xlims[1]

    # make mask for spectrogram
    f_mask = np.arange(len(grid.fill_freqs))[
        (grid.fill_freqs >= f0 + 5) & (grid.fill_freqs <= f1 + 5)
    ]
    t_mask = np.arange(len(grid.fill_times))[
        (grid.fill_times >= t0) & (grid.fill_times <= t1)
    ]

    # plot spectrogram
    ax.imshow(
        tf.powerspectrum.decibel(
            grid.fill_spec[f_mask[0] : f_mask[-1], t_mask[0] : t_mask[-1]][::-1]
        ),
        extent=[np.min(newtime), np.max(newtime), f0, f1],
        vmin=-100,
        vmax=-50,
        **style.fine_spec,
    )


# plot traces
#plot_finespec(ax1, idxs1, dyad1, newtime1, grid1, s)
#ax1.plot(newtime1, dyad1.fund_id1[idxs1[0]:idxs1[1]], **s.id1_slim)
#ax1.plot(newtime1, dyad1.fund_id2[idxs1[0]:idxs1[1]], **s.id2_slim)

plot_finespec(ax2, idxs2, dyad2, newtime2, grid2, s)
ax2.plot(newtime2, dyad2.fund_id1[idxs2[0]:idxs2[1]], **s.id1_slim)
ax2.plot(newtime2, dyad2.fund_id2[idxs2[0]:idxs2[1]], **s.id2_slim)
# 
plot_finespec(ax1, idxs4, dyad4, newtime4, grid4, s)
ax1.plot(newtime4, dyad4.fund_id1[idxs4[0]:idxs4[1]], **s.id1_slim)
ax1.plot(newtime4, dyad4.fund_id2[idxs4[0]:idxs4[1]], **s.id2_slim)

plot_finespec(ax3, idxs3, dyad3, newtime3, grid3, s)
ax3.plot(newtime3, dyad3.fund_id1[idxs3[0]:idxs3[1]], **s.id1_slim)
ax3.plot(newtime3, dyad3.fund_id2[idxs3[0]:idxs3[1]], **s.id2_slim)

fig.savefig('../poster/figs/selected_modulations.pdf')